In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gears import PertData, GEARS
import numpy as np
import pandas as pd
from tqdm import tqdm
import scanpy as sc
import anndata
from scipy.sparse import csr_matrix

In [3]:
data_dir = '/home/vinas/sc_perturbation_benchmark/data/tian_CRISPRi_2021_scperturb'

#### Download data

Download data from:
```
https://zenodo.org/records/7041849/files/TianKampmann2021_CRISPRi.h5ad?download=1
```

#### Process data

In [5]:
adata = anndata.read_h5ad(f'{data_dir}/TianKampmann2021_CRISPRi.h5ad')
adata

AnnData object with n_obs × n_vars = 32300 × 33538
    obs: 'guide_id', 'perturbation', 'tissue_type', 'celltype', 'cancer', 'disease', 'perturbation_type', 'organism', 'batch', 'nperts', 'ngenes', 'ncounts', 'percent_mito', 'percent_ribo', 'percent_hemo'
    var: 'ensembl_id', 'ncounts', 'ncells'

In [6]:
genes = adata.obs['perturbation'].unique().tolist()

for i, gene in enumerate(genes):
    if gene not in adata.var_names.tolist():
        print(gene)

TMEM55A
ATP5C1
control
ATP5H


In [7]:
target_genes = adata.obs['perturbation'].tolist()
target_genes[target_genes=='TMEM55A']='PIP4P2'
target_genes[target_genes=='ATP5C1']='ATP5F1C'
target_genes[target_genes=='ATP5H']='ATP5PD'
adata.obs['perturbation'] = target_genes

In [8]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=5000, subset=False)

In [9]:
adata.var['gene_name'] = adata.var.index
adata.obs['gene'] = adata.obs['perturbation']
hvg_flag_ = adata.var['highly_variable'].values
gene_flag_ = adata.var['gene_name'].isin(adata.obs['gene'].values).values
select_flag_ = np.logical_or(hvg_flag_, gene_flag_)
condition_flag_ = adata.obs['gene'].isin(adata.var['gene_name'].values.tolist()+['control']).values
adata_subset = adata[condition_flag_, select_flag_]

In [10]:
pert_counts = adata_subset.obs['gene'].value_counts()
pert_counts

gene
control    437
TAF1       375
DNAJC6     362
CREBBP     323
PAXIP1     305
          ... 
HSPA5       81
KIF1B       73
REEP2       72
TARDBP      45
ATP5PD       1
Name: count, Length: 183, dtype: int64

In [11]:
# Discard ATP5PD pert (GEARS needs > 1 sample to calculate statistics)
m = pert_counts > 1
keep_perts = m[m].index.values
adata_subset = adata_subset[adata_subset.obs['gene'].isin(keep_perts)]

In [12]:
# Prepare data in GEARS format
adata_subset.obs['condition'] = [i+'+ctrl' for i in adata_subset.obs['gene'].values]
adata_subset.obs['condition'] = adata_subset.obs['condition'].replace({'control+ctrl': 'ctrl'})
adata_subset.obs['cell_type'] = 'iPSC'
len(adata_subset.obs['condition'].unique())

182

In [13]:
out_dir = f'{data_dir}'
pert_data = PertData(out_dir)
pert_data.new_data_process(dataset_name = 'CRISPRi', adata = adata_subset)

Downloading...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.46M/9.46M [00:00<00:00, 9.60MiB/s]
Downloading...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 559k/559k [00:00<00:00, 1.44MiB/s]
Creating pyg object for each cell in the data...
Creating dataset file...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 182/182 [34